카카오 이미지 검색결과 수집하기

###### 패션 잡지에 넣을 봄을 주제로 한 옷차림을 알아보고자 한다. 해당 주제로 검색된 이미지를 수집하여라

kakao developers 참고

In [ ]:
# 필요한 모듈 설정
import requests
# json 데이터
import json
# 데이터 시각화 처리
from pandas import DataFrame
# 운영 체제와 관련된 기능들을 제공
import os

In [ ]:
# 접속할 URL 설정

urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"

In [ ]:
# 요청변수
query = '봄 패션'
page = 1
size = 80
key = '3b23dfd1bf5a535f83c2792589d13d21'

In [ ]:
# 브라우저 버전 정보
userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

In [ ]:
# 접속객체 생성
session = requests.Session()

# 접속 객체에 부가정보 (header) 삽입
session.headers.update({
    "Authorization":"KakaoAK %s" %key
})

이론상 4000장의 사진을 긁어오기 때문에 그 이하의 정보를 저장하고 싶으면 반복문에서 break를 걸어준다

In [ ]:
# 다음 페이지 존재 여부(False인 동안은 while문 반복 유도)
isEnd = False

# 1페이지부터 시작
page = 1

# 저장된 이미지 카운트
count = 0

while (not isEnd):
    # 여기는 이전(05, 06)코드
    url = urlFmt.format(query=query,page=page,size=size)
    url
    # 생성한 접속객체를 활용하여 API에 접속
    r = session.get(url)

    # 접속 실패 시 에러 메시지
    if r.status_code != 200:
        msg = "[%d Error] %s 에러 발생" % (r.status_code, r.reason)
        raise Exception(msg)
    
    # 인코딩 형식 지정
    r.encoding = "utf-8"
    # 가져온 문자열을 딕셔너리로 변환
    data = json.loads(r.text)
    # data
    # 여기까지 이전(05, 06)코드

    # 검색 결과를 가지고 있는 딕셔너리 포함 리스트 추출
    searchList = data ['documents']

    # 증감식에 해당하는 isEnd 변수 추출
    isEnd = bool(data['meta']['is_end'])
    page+=1
    print("isEnd=",isEnd,"/ 다음페이지=",page)

    # 이미지 저장 폴더 생성
    # 현재 폴더에 query로된 이름의 폴더가 없으면
    # query를 사용해 폴더 생성
    # 상대경로는 그냥 이름(현재 폴더)
    # 절대경로는 C 또는 //로 시작하는 폴더 이름
    if os.path.exists(query) == False:
        os.mkdir(query)
    
    # 이미지 저장
    for i, v in enumerate(searchList):
        # 숫자 카운트
        count += 1

        # 이미지 URL 추출
        docUrl = v['doc_url']
        imageUrl = v['image_url']
        print(imageUrl)

        # 접속객체에 부가정보(header) 삽입하기
        session.headers.update({
            "Authorization" : "KakaoAK %s" % key,
            "Referer" : docUrl,
            "User-Agen": userAgent
        })

        # 텍스트 이외의 파일 형식을 가져오기 위해 stream 파라미터 추가
        try:
            # 다운로드를 위해 stram=True
            r = session.get(imageUrl, stream=True)
        except:
            continue
        # 다운로드에 실패한 경우 다음 이미지로 이동
        if r.status_code != 200:
            continue

        # 파일이름 설정
        # query폴더에 nnnnn.png로 저장
        fileName = "%s/%05d.png" % (query, count)

        with open(fileName, 'wb') as f:
            # 통신으로 가져온 내용의 바이너리(기계어)를 직접 추출
            f.write(r.raw.read())
            print(fileName, "(이)가 저장")
print("종료")